In [1]:
import numpy as np

def count_results(B, GTB, TP, TN, FP, FN):
    TP_M = np.logical_and((B == 255), (GTB == 255)) 
    TP_S = np.sum ( TP_M ) 
    TP += TP_S 

    TN_M = np.logical_and((B == 0), (GTB == 0)) 
    TN_S = np.sum (TN_M) 
    TN += TN_S 

    FP_M = np.logical_and((B == 255), (GTB == 0))
    FP_S = np.sum(FP_M)
    FP += FP_S

    FN_M = np.logical_and((B == 0), (GTB == 255))
    FN_S = np.sum(FN_M)
    FN += FN_S

    return TP, TN, FP, FN

In [ ]:
import cv2
import numpy as np

def find_bg_based_on_buffor(folder, start_index, buffor_size):
    I = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    YY, XX = I.shape    
    
    BUF = np.zeros((YY, XX, buffor_size), np.uint8)
    for i in range(start_index, start_index + buffor_size):
        img = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        BUF[:, :, i - start_index] = img

    BG = np.zeros((YY, XX))

    for y in range(YY):
        for x in range(XX):
            unique, counts = np.unique(BUF[y, x, :], return_counts=True)
            d = dict(zip(unique, counts))
            BG[y, x] = max(d, key=d.get)

    return BG

def segmentation_conservatice_better(folder, type, alfa):    
    I = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype('float64')

    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    
    start_mark = 1000

    BG = find_bg_based_on_buffor(folder, roi_start + start_mark, 30)
    last_b = np.zeros(BG.shape)

    for i in range (roi_start + start_mark, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('float64')

        valid = (last_b == 0)

        if type == 'mean':
            BG[valid] = alfa*IG[valid] + (1-alfa)*BG[valid]

        elif type == 'median':
            BG[valid] = BG[valid]
            BG[(BG < IG) & (last_b == 0)] += 1
            BG[(BG > IG) & (last_b == 0)] -= 1
    
        scene = cv2.absdiff(BG.astype('uint8'), IG.astype('uint8'))
        blur = cv2.GaussianBlur(scene, (3, 3), 0)
        ret, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        B = thresh

        I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP, TN, FP, FN = count_results(B, GTB, TP, TN, FP, FN)

        cv2.imshow("I", B)
        cv2.waitKey(10)
          
        last_b = B
        
    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nType: {type}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

def getRandomNeighbour(x, maxSize):
    val = None
    if x == 0:
        val = 1
    elif x == maxSize - 1:
        val = maxSize - 2
    else:
        rand = np.random.randint(0, 1)
        if rand == 0:
            val = x - 1
        else:
            val = x + 1
    return val

def vibe(image, samples):

    N = 20
    R = 20
    min_samples = 2
    subsamples = 16
    height, width = image.shape

    segMap = np.zeros((height, width))
    background = 0
    foreground = 255

    for y in range(height):
        for x in range(width):
            count = 0
            index = 0
            dist = 0
            while count < min_samples and index < N:
                dist = np.sqrt((image[y, x] - samples[y, x, index])**2) # maybe wrong
                if dist < R:
                    count += 1
                index += 1

            if count >= min_samples:
                segMap[y, x] = background
                rand = np.random.randint(0, subsamples - 1)
                if rand == 0:
                    rand = np.random.randint(0, N - 1)
                    samples[y, x, rand] = image[y, x]

                rand = np.random.randint(0, subsamples - 1)
                if rand == 0:
                    xNg = getRandomNeighbour(x, width)
                    yNg = getRandomNeighbour(y, height)
                    rand = np.random.randint(0, N - 1)

                    samples[yNg, xNg, rand] = image[y, x]
            else:
                segMap[y, x] = foreground

    return segMap, samples

In [3]:
def segmentation_vibe(folder):    
    BG = cv2.imread(f'../lab2/{folder}/input/in000001.jpg')
    BG = cv2.cvtColor(BG, cv2.COLOR_BGR2GRAY).astype('float64')
    # TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'../lab2/{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)

    N = 20
    height, width = BG.shape
    samples = np.zeros((height, width, N))

    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'../lab2/{folder}/input/in%06d.jpg' % i)
        IG = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('float64')

        B, samples = vibe(IG, samples)
        # print(B)
        # I_ground = cv2.imread(f'../lab2/{folder}/groundtruth/gt%06d.png' % i)
        # GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        # TP, TN, FP, FN = count_results(B, GTB, TP, TN, FP, FN)
        
        cv2.imshow("I", B)
        cv2.waitKey(10)

    # precission = TP / (TP + FP)
    # recall = TP / (TP + FN)
    # f1 = 2 * precission * recall / (precission + recall)
    # print(f"Clip Name: {folder}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()

segmentation_vibe('pedestrian')

In [ ]:
import cv2
import numpy as np

class VIBE:
    def __init__(self, num_samples=20, radius=20, min_matches=2, subsampling=16):
        self.num_samples = num_samples
        self.radius = radius
        self.min_matches = min_matches
        self.subsampling = subsampling
        self.background_samples = None

    def initialize_background_model(self, frame):
        self.background_samples = np.zeros((frame.shape[0], frame.shape[1], self.num_samples), dtype=np.uint8)
        for i in range(self.num_samples):
            x = np.random.randint(0, frame.shape[0])
            y = np.random.randint(0, frame.shape[1])
            self.background_samples[:,:,i] = frame[x,y]

    def update_background_model(self, frame):
        matches = np.zeros_like(frame, dtype=np.uint8)
        for i in range(self.num_samples):
            diff = np.abs(frame.astype(int) - self.background_samples[:,:,i].astype(int))
            matches += (diff < self.radius).astype(np.uint8)
        
        foreground = matches < self.min_matches
        if np.random.randint(0, self.subsampling) == 0:
            self.background_samples[:,:,np.random.randint(0, self.num_samples)] = frame.copy()
            
        # if np.random.randint(0, self.subsampling) == 0:

        return foreground

if __name__ == "__main__":
    vibe = VIBE(num_samples=20, radius=20, min_matches=2)
    cap = cv2.VideoCapture('pedestrians_input.mp4')  # Replace 'your_video_file.mp4' with your video file

    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret or frame is None:
            break
        
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if vibe.background_samples is None:
            vibe.initialize_background_model(frame_gray)
        else:
            foreground_mask = vibe.update_background_model(frame_gray)
            foreground_mask = foreground_mask.astype('uint8')
            cv2.imshow('Foreground', foreground_mask*255)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()